In [4]:
import pandas as pd
import psycopg2
import pycountry
import gettext
import translators
import pytz
from tzwhere import tzwhere
import datetime

In [44]:
russian = gettext.translation('iso3166', pycountry.LOCALES_DIR, languages=['ru'])
russian.install()

In [46]:
_('Germany')

'Германия'

In [104]:
translation = translators.bing('Harbin', to_language='ru')
translation

In [12]:
cities = pd.read_csv('city.csv', sep=';', encoding='cp1251')
countries = pd.read_csv('country.csv', sep=';', encoding='cp1251')
regions = pd.read_csv('region.csv', sep=';', encoding='cp1251')
airports_csv = pd.read_csv('airports.csv')

In [225]:
countries = countries[['country_id', 'name']]
countries

,country_id,name
0,3159,Россия
1,4,Австралия
2,63,Австрия
3,81,Азербайджан
4,173,Ангуилья
...,...,...
101,3661568,ЮАР
102,11002,Югославия
103,11014,Южная Корея
104,582106,Ямайка


In [226]:
cities = cities[['city_id', 'name', 'country_id']]
cities

,city_id,name,country_id
0,4400,Москва,3159
1,4313,Абрамцево,3159
2,4314,Алабино,3159
3,4315,Апрелевка,3159
4,4316,Архангельское,3159
...,...,...,...
10964,11359,Хаги,11060
10965,11360,Хофу,11060
10966,11361,Шимоносеки,11060
10967,11362,Ямагучи,11060


In [36]:
large_airports = airports_csv[
    (airports_csv['type'] != 'closed')&
    (airports_csv['type'] != 'heliport')&
    (airports_csv['type'] != 'small_airport')&
    (airports_csv['type'] != 'seaplane_base')&
    (airports_csv['type'] != 'balloonport')&
    (airports_csv['type'] != 'medium_airport')
    ]

In [79]:
def getCountryName(code):
    try:
        return _(pycountry.countries.get(alpha_2=code).name)
    except:
        print('ошибка с кодом:', code)
        return 'nan'

In [187]:
def translate(_str):
    try:
        return translators.bing(_str, to_language='ru')
    except:
        try:
            return translators.google(_str, to_language='ru')
        except:
            try:
                return translators.tencent(_str, to_language='ru')
            except:
                print('ошибка с:', _str)
                return _str

In [135]:
large_airports = large_airports[
    (large_airports['iata_code'].notnull())
    ]

In [ ]:
large_airports['country_name'] = large_airports['iso_country'].apply(getCountryName)
large_airports['city_name'] = large_airports['municipality'].apply(translate)

In [ ]:
airports_data = large_airports[['id', 'name', 'latitude_deg', 'longitude_deg', 'country_name', 'city_name']]
airports_data

In [227]:
airports_countries = pd.merge(airports_data, countries, left_on="country_name", right_on="name")
airports_countries.drop(['name_y'], axis=1, inplace=True)
airports_countries.rename(columns={'name_x': 'airport_name'}, inplace=True)
airports_countries

,id,airport_name,latitude_deg,longitude_deg,country_name,city_name,country_id
0,1717,Edmonton International Airport,53.309700,-113.580002,Канада,Эдмонтон,2172
1,1770,Halifax / Stanfield International Airport,44.880798,-63.508598,Канада,Галифакс,2172
2,1808,Kelowna International Airport,49.956100,-119.377998,Канада,Келоуна,2172
3,1840,Ottawa Macdonald-Cartier International Airport,45.322498,-75.669197,Канада,Оттава,2172
4,1864,Quebec Jean Lesage International Airport,46.791100,-71.393303,Канада,Квебек,2172
...,...,...,...,...,...,...,...
239,27236,Ürümqi Diwopu International Airport,43.907101,87.474197,Китай,Урумчи,2374
240,27238,Harbin Taiping International Airport,45.623402,126.250000,Китай,Харбин,2374
241,27242,Dalian Zhoushuizi International Airport,38.965698,121.539001,Китай,"Ганьцзинцзы, Далянь",2374
242,27243,Shenyang Taoxian International Airport,41.639801,123.483002,Китай,"Хуннань, Шэньян",2374


In [231]:
airports_countries_cities = pd.merge(airports_countries, cities, left_on=['city_name', 'country_id'], right_on=['name', 'country_id'])
airports_countries_cities.drop(['name'], axis=1, inplace=True)
airports_countries_cities

,id,airport_name,latitude_deg,longitude_deg,country_name,city_name,country_id,city_id
0,1717,Edmonton International Airport,53.309700,-113.580002,Канада,Эдмонтон,2172,2180
1,1770,Halifax / Stanfield International Airport,44.880798,-63.508598,Канада,Галифакс,2172,2233
2,1840,Ottawa Macdonald-Cartier International Airport,45.322498,-75.669197,Канада,Оттава,2172,2274
3,1864,Quebec Jean Lesage International Airport,46.791100,-71.393303,Канада,Квебек,2172,2212
4,1928,Montreal / Pierre Elliott Trudeau Internationa...,45.470600,-73.740799,Канада,Монреаль,2172,2215
...,...,...,...,...,...,...,...,...
132,27193,Taiyuan Wusu Airport,37.746899,112.627998,Китай,Тайюань,2374,2429
133,27200,Wuhan Tianhe International Airport,30.783800,114.208000,Китай,Ухань,2374,2399
134,27221,Nanjing Lukou International Airport,31.735032,118.865949,Китай,Нанкин,2374,2418
135,27236,Ürümqi Diwopu International Airport,43.907101,87.474197,Китай,Урумчи,2374,2377


In [232]:
with pd.ExcelWriter('airports_countries_cities.xlsx') as writer: 
    airports_countries_cities.to_excel(writer)

перевёл отдельно)

In [234]:
airports_countries_cities = pd.read_excel('airports_countries_cities.xlsx')
airports_countries_cities

,id,airport_name,latitude_deg,longitude_deg,country_name,city_name,country_id,city_id
0,1717,Международный аэропорт Эдмонтона,53.309700,-113.580002,Канада,Эдмонтон,2172,2180
1,1770,Международный аэропорт Галифакс/Стэнфилд,44.880798,-63.508598,Канада,Галифакс,2172,2233
2,1840,Международный аэропорт Оттавы Макдональд-Картье,45.322498,-75.669197,Канада,Оттава,2172,2274
3,1864,Международный аэропорт Квебек Жан Лесаж,46.791100,-71.393303,Канада,Квебек,2172,2212
4,1928,Монреаль / Международный аэропорт имени Пьера ...,45.470600,-73.740799,Канада,Монреаль,2172,2215
...,...,...,...,...,...,...,...,...
132,27193,Аэропорт Тайюань Усу,37.746899,112.627998,Китай,Тайюань,2374,2429
133,27200,Международный аэропорт Ухань Тяньхэ,30.783800,114.208000,Китай,Ухань,2374,2399
134,27221,Международный аэропорт Нанкин Лукоу,31.735032,118.865949,Китай,Нанкин,2374,2418
135,27236,Международный аэропорт Урюмчи Дивопу,43.907101,87.474197,Китай,Урумчи,2374,2377


In [249]:
def getTimezone(lat, long):
    _tzwhere = tzwhere.tzwhere()
    timezone_str = _tzwhere.tzNameAt(lat, long)
    return timezone_str

In [ ]:
airports_countries_cities['timezone'] = airports_countries_cities.apply(lambda row: getTimezone(row['latitude_deg'], row['longitude_deg']), axis=1)

In [252]:
airports_countries_cities

,id,airport_name,latitude_deg,longitude_deg,country_name,city_name,country_id,city_id,timezone
0,1717,Международный аэропорт Эдмонтона,53.309700,-113.580002,Канада,Эдмонтон,2172,2180,America/Edmonton
1,1770,Международный аэропорт Галифакс/Стэнфилд,44.880798,-63.508598,Канада,Галифакс,2172,2233,America/Halifax
2,1840,Международный аэропорт Оттавы Макдональд-Картье,45.322498,-75.669197,Канада,Оттава,2172,2274,America/Toronto
3,1864,Международный аэропорт Квебек Жан Лесаж,46.791100,-71.393303,Канада,Квебек,2172,2212,America/Montreal
4,1928,Монреаль / Международный аэропорт имени Пьера ...,45.470600,-73.740799,Канада,Монреаль,2172,2215,America/Montreal
...,...,...,...,...,...,...,...,...,...
132,27193,Аэропорт Тайюань Усу,37.746899,112.627998,Китай,Тайюань,2374,2429,Asia/Shanghai
133,27200,Международный аэропорт Ухань Тяньхэ,30.783800,114.208000,Китай,Ухань,2374,2399,Asia/Shanghai
134,27221,Международный аэропорт Нанкин Лукоу,31.735032,118.865949,Китай,Нанкин,2374,2418,Asia/Shanghai
135,27236,Международный аэропорт Урюмчи Дивопу,43.907101,87.474197,Китай,Урумчи,2374,2377,Asia/Urumqi


In [299]:
airports_table = airports_countries_cities[['id', 'airport_name', 'latitude_deg', 'longitude_deg', 'timezone', 'city_id']]
airports_table.rename(columns={'id': 'airport_id', 'airport_name': 'name', 'latitude_deg': 'latitude', 'longitude_deg': 'longitude'}, inplace=True)
airports_table

C:\Users\rouol\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,airport_id,name,latitude,longitude,timezone,city_id
0,1717,Международный аэропорт Эдмонтона,53.309700,-113.580002,America/Edmonton,2180
1,1770,Международный аэропорт Галифакс/Стэнфилд,44.880798,-63.508598,America/Halifax,2233
2,1840,Международный аэропорт Оттавы Макдональд-Картье,45.322498,-75.669197,America/Toronto,2274
3,1864,Международный аэропорт Квебек Жан Лесаж,46.791100,-71.393303,America/Montreal,2212
4,1928,Монреаль / Международный аэропорт имени Пьера ...,45.470600,-73.740799,America/Montreal,2215
...,...,...,...,...,...,...
132,27193,Аэропорт Тайюань Усу,37.746899,112.627998,Asia/Shanghai,2429
133,27200,Международный аэропорт Ухань Тяньхэ,30.783800,114.208000,Asia/Shanghai,2399
134,27221,Международный аэропорт Нанкин Лукоу,31.735032,118.865949,Asia/Shanghai,2418
135,27236,Международный аэропорт Урюмчи Дивопу,43.907101,87.474197,Asia/Urumqi,2377


In [302]:
airports_table[:17]

,airport_id,name,latitude,longitude,timezone,city_id
0,1717,Международный аэропорт Эдмонтона,53.309700,-113.580002,America/Edmonton,2180
1,1770,Международный аэропорт Галифакс/Стэнфилд,44.880798,-63.508598,America/Halifax,2233
2,1840,Международный аэропорт Оттавы Макдональд-Картье,45.322498,-75.669197,America/Toronto,2274
3,1864,Международный аэропорт Квебек Жан Лесаж,46.791100,-71.393303,America/Montreal,2212
4,1928,Монреаль / Международный аэропорт имени Пьера ...,45.470600,-73.740799,America/Montreal,2215
5,1941,Международный аэропорт Ванкувера,49.193901,-123.183998,America/Vancouver,2183
6,1946,Виннипег / Международный аэропорт имени Джеймс...,49.910000,-97.239899,America/Winnipeg,2231
7,1973,Международный аэропорт Калгари,51.113899,-114.019997,America/Edmonton,2175
8,1986,Международный аэропорт Сент-Джонс,47.618599,-52.751900,America/St_Johns,2241
9,1990,Международный аэропорт имени Лестера Б. Пирсона,43.677200,-79.630600,America/Toronto,2287


In [5]:
conn = psycopg2.connect(
    host="localhost",
    database="skyscanner",
    user="postgres",
    password="")

In [6]:
cur = conn.cursor()
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

('PostgreSQL 10.19, compiled by Visual C++ build 1800, 64-bit',)


In [10]:
curs = conn.cursor()
curs.execute("ROLLBACK")
conn.commit()

In [335]:
countries

,country_id,name
0,3159,Россия
1,4,Австралия
2,63,Австрия
3,81,Азербайджан
4,173,Ангуилья
...,...,...
101,3661568,ЮАР
102,11002,Югославия
103,11014,Южная Корея
104,582106,Ямайка


In [347]:
cursor = conn.cursor()
for index, row in countries.iterrows():
    cursor.execute("INSERT INTO countries (country_id, name) VALUES(%s, %s)", (row['country_id'], row['name']))
conn.commit()

In [324]:
cities

,city_id,name,country_id
0,4400,Москва,3159
1,4313,Абрамцево,3159
2,4314,Алабино,3159
3,4315,Апрелевка,3159
4,4316,Архангельское,3159
...,...,...,...
10964,11359,Хаги,11060
10965,11360,Хофу,11060
10966,11361,Шимоносеки,11060
10967,11362,Ямагучи,11060


In [349]:
cursor = conn.cursor()
for index, row in cities.iterrows():
    cursor.execute("INSERT INTO cities (city_id, name, country_id) VALUES(%s, %s, %s)", (row['city_id'], row['name'], row['country_id']))
conn.commit()

In [378]:
cursor = conn.cursor()
c = 0
for index, row in airports_table.iterrows():
    if row['timezone'] != None:
        cursor.execute("INSERT INTO airports (airport_id, name, latitude, longitude, timezone, city_id) VALUES(%s, %s, %s, %s, %s, %s)",
            (c, row['name'], row['latitude'], row['longitude'], row['timezone'], row['city_id']))
        c+=1
conn.commit()

In [ ]:
airlines = pd.read_excel('airlines_rus.xlsx')
airlines['airline_id'] = range(1, len(airlines) + 1)
airlines.rename(columns={'Авиакомпания': 'name'}, inplace=True)
airlines['contact_info'] = ''
airlines

In [13]:
airlines = pd.read_csv('Airlines.csv')
airlines['airline_id'] = range(1+32, len(airlines) + 1 + 32)
airlines.rename(columns={'#Name': 'name'}, inplace=True)
airlines['contact_info'] = ''
airlines

,name,IATA,ICAO,Country,Active,airline_id,contact_info
0,Indonesa Air Aisa X,ZM,IDX,Indonesia,Y,33,
1,Alghanim,ZL,KYA,United States,Y,34,
2,Norwegian Air International,ZK,IBK,Norway,Y,35,
3,Boliviana de Aviacion,ZI,BOV,Bolivia,Y,36,
4,AtlasGlobal Ukraine,ZH,UJX,Ukraine,Y,37,
...,...,...,...,...,...,...,...
505,Sefofane Air Charters,BN,SFE,South Africa,N,538,
506,Austrian Airlines,OS,AUA,Austria,Y,539,
507,Turkish Airlines,TH,THY,Turkey,Y,540,
508,Lufthansa,MH,DLH,Germany,Y,541,


In [14]:
airlines_table = airlines[['airline_id', 'name', 'contact_info']]
airlines_table

,airline_id,name,contact_info
0,33,Indonesa Air Aisa X,
1,34,Alghanim,
2,35,Norwegian Air International,
3,36,Boliviana de Aviacion,
4,37,AtlasGlobal Ukraine,
...,...,...,...
505,538,Sefofane Air Charters,
506,539,Austrian Airlines,
507,540,Turkish Airlines,
508,541,Lufthansa,


In [15]:
cursor = conn.cursor()
for index, row in airlines_table.iterrows():
    cursor.execute("INSERT INTO airlines (airline_id, name, contact_info) VALUES(%s, %s, %s)",
    (row['airline_id'], row['name'], row['contact_info']))
conn.commit()

In [57]:
airline_id = 23 # Аэрофлот
model = 'Airbus A320'
flight_num = 1636
status = 'available'
departure_airport_id = 129 # Москва
arrive_airport_id = 133 # Симферополь
actual_departure_time = None
actual_arrival_time = None
base_flight_id = int(str(airline_id) + str(flight_num)) * 1000
base_aircraft_id = base_flight_id
base_departure_time = datetime.datetime(2022, 3, 17, 9, 30, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_arrival_time = datetime.datetime(2022, 3, 17, 10, 20, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_flight_id, base_departure_time, base_arrival_time

(231636000,
 datetime.datetime(2022, 3, 17, 9, 30, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>),
 datetime.datetime(2022, 3, 17, 10, 20, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>))

In [18]:
airline_id = 23 # Аэрофлот
model = 'Airbus A320'
flight_num = 1637
status = 'available'
departure_airport_id = 133 # Симферополь
arrive_airport_id = 129 # Москва
actual_departure_time = None
actual_arrival_time = None
base_flight_id = int(str(airline_id) + str(flight_num)) * 1000
base_aircraft_id = base_flight_id
base_departure_time = datetime.datetime(2022, 3, 17, 11, 10, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_arrival_time = datetime.datetime(2022, 3, 17, 14, 0, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_flight_id, base_departure_time, base_arrival_time

(231637000,
 datetime.datetime(2022, 3, 17, 11, 10, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>),
 datetime.datetime(2022, 3, 17, 14, 0, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>))

In [24]:
airline_id = 22 # S7 Airlines (Сибирь)
model = 'Boeing 737-800'
flight_num = 2008
status = 'available'
departure_airport_id = 133 # Симферополь
arrive_airport_id = 129 # Москва
actual_departure_time = None
actual_arrival_time = None
base_flight_id = int(str(airline_id) + str(flight_num)) * 1000
base_aircraft_id = base_flight_id
base_departure_time = datetime.datetime(2022, 3, 17, 12, 00, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_arrival_time = datetime.datetime(2022, 3, 17, 14, 40, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_flight_id, base_departure_time, base_arrival_time

(222008000,
 datetime.datetime(2022, 3, 17, 12, 0, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>),
 datetime.datetime(2022, 3, 17, 14, 40, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>))

In [29]:
airline_id = 15 # Северный Ветер
model = 'Airbus А321'
flight_num = 134
status = 'available'
departure_airport_id = 133 # Симферополь
arrive_airport_id = 129 # Москва
actual_departure_time = None
actual_arrival_time = None
base_flight_id = int(str(airline_id) + '0' + str(flight_num)) * 1000
base_aircraft_id = base_flight_id
base_departure_time = datetime.datetime(2022, 3, 17, 19, 50, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_arrival_time = datetime.datetime(2022, 3, 17, 22, 20, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_flight_id, base_departure_time, base_arrival_time

(150134000,
 datetime.datetime(2022, 3, 17, 19, 50, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>),
 datetime.datetime(2022, 3, 17, 22, 20, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>))

In [7]:
# в марте
airline_id = 55 # Finnair
model = 'Airbus A319'
flight_num = 714
status = 'available'
departure_airport_id = 129 # Москва
arrive_airport_id = 20 # Хельсинки
actual_departure_time = None
actual_arrival_time = None
base_flight_id = int(str(airline_id) + '0' + str(flight_num)) * 1000
base_aircraft_id = base_flight_id
base_departure_time = datetime.datetime(2022, 3, 17, 21, 0, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_arrival_time = datetime.datetime(2022, 3, 17, 22, 20, 0, tzinfo=pytz.timezone('Europe/Helsinki'))
base_flight_id, base_departure_time, base_arrival_time

(550714000,
 datetime.datetime(2022, 3, 17, 21, 0, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>),
 datetime.datetime(2022, 3, 17, 22, 20, tzinfo=<DstTzInfo 'Europe/Helsinki' LMT+1:40:00 STD>))

In [ ]:
airline_id = 55 # Finnair
model = 'Airbus A319'
flight_num = 714
status = 'available'
departure_airport_id = 129 # Москва
arrive_airport_id = 20 # Хельсинки
actual_departure_time = None
actual_arrival_time = None
base_flight_id = int(str(airline_id) + '0' + str(flight_num)) * 1000
base_aircraft_id = base_flight_id
base_departure_time = datetime.datetime(2021, 12, 20, 21, 0, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_arrival_time = datetime.datetime(2021, 12, 20, 22, 20, 0, tzinfo=pytz.timezone('Europe/Helsinki'))
base_flight_id, base_departure_time, base_arrival_time

In [51]:
airline_id = 55 # Finnair
model = 'Atr 72'
flight_num = 702
status = 'available'
departure_airport_id = 130 # СПБ
arrive_airport_id = 20 # Хельсинки
actual_departure_time = None
actual_arrival_time = None
base_flight_id = int(str(airline_id) + '0' + str(flight_num)) * 1000
base_aircraft_id = base_flight_id
base_departure_time = datetime.datetime(2021, 12, 20, 21, 0, 0, tzinfo=pytz.timezone('Europe/Moscow'))
base_arrival_time = datetime.datetime(2021, 12, 20, 22, 20, 0, tzinfo=pytz.timezone('Europe/Helsinki'))
base_flight_id, base_departure_time, base_arrival_time

(550702000,
 datetime.datetime(2021, 12, 20, 21, 0, tzinfo=<DstTzInfo 'Europe/Moscow' LMT+2:30:00 STD>),
 datetime.datetime(2021, 12, 20, 22, 20, tzinfo=<DstTzInfo 'Europe/Helsinki' LMT+1:40:00 STD>))

In [52]:
cursor = conn.cursor()
for flight_id in range(base_flight_id + 1, base_flight_id + 30):
    aircraft_id = flight_id
    cursor.execute("INSERT INTO aircrafts (aircraft_id, model) VALUES (%s, %s)", ((aircraft_id, model)))
conn.commit()

In [11]:
cursor = conn.cursor()
for flight_id in range(base_flight_id + 30, base_flight_id + 60):
    aircraft_id = flight_id
    cursor.execute("INSERT INTO aircrafts (aircraft_id, model) VALUES (%s, %s)", ((aircraft_id, model)))
conn.commit()

In [53]:
cursor = conn.cursor()
departure_time = base_departure_time
arrival_time = base_arrival_time
for flight_id in range(base_flight_id + 1, base_flight_id + 30):
    aircraft_id = flight_id
    # print(flight_id, departure_time, arrival_time, status, actual_departure_time, actual_arrival_time, departure_airport_id, arrive_airport_id, airline_id, aircraft_id)
    cursor.execute("INSERT INTO flights\
        (flight_id, departure_time, arrival_time, status, actual_departure_time, actual_arrival_time, departure_airport_id, arrive_airport_id, airline_id, aircraft_id) \
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", ((flight_id, departure_time, arrival_time, status, actual_departure_time, actual_arrival_time, departure_airport_id, arrive_airport_id, airline_id, aircraft_id)))
    departure_time += datetime.timedelta(days=1)
    arrival_time += datetime.timedelta(days=1)
conn.commit()

In [12]:
cursor = conn.cursor()
departure_time = base_departure_time
arrival_time = base_arrival_time
for flight_id in range(base_flight_id + 30, base_flight_id + 60):
    aircraft_id = flight_id
    # print(flight_id, departure_time, arrival_time, status, actual_departure_time, actual_arrival_time, departure_airport_id, arrive_airport_id, airline_id, aircraft_id)
    cursor.execute("INSERT INTO flights\
        (flight_id, departure_time, arrival_time, status, actual_departure_time, actual_arrival_time, departure_airport_id, arrive_airport_id, airline_id, aircraft_id) \
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", ((flight_id, departure_time, arrival_time, status, actual_departure_time, actual_arrival_time, departure_airport_id, arrive_airport_id, airline_id, aircraft_id)))
    departure_time += datetime.timedelta(days=1)
    arrival_time += datetime.timedelta(days=1)
conn.commit()

In [38]:
fares = {
    'Эконом': {
        'Эконом Плоский': 6080
        },
    'Бизнес': {
        'Бизнес Базовый': 50786,
        'Бизнес Оптимум': 59186,
        'Бизнес Максимум': 73186
        }
}
_status = 'available'

In [13]:
#Finn МСК-Х
fares = {
    'Эконом': {
        'ECONOMY LIGHT': 7710,
        'ECONOMY CLASSIC': 10200,
        'ECONOMY FLEX': 13520
        },
    'Бизнес': {
        'BUSINESS LIGHT': 30450,
        'BUSINESS CLASSIC': 35845,
        'BUSINESS FLEX': 42485
        }
}
_status = 'available'

In [54]:
#Finn СПБ-Х
fares = {
    'Эконом': {
        'ECONOMY LIGHT': 8797,
        'ECONOMY CLASSIC': 11287,
        'ECONOMY FLEX': 14607
        }
    # 'Бизнес': {
    #     'BUSINESS LIGHT': 30450,
    #     'BUSINESS CLASSIC': 35845,
    #     'BUSINESS FLEX': 42485
    #     }
}
_status = 'available'

In [32]:
fares = {
    'Эконом': {
        'Эконом Лайт': 3906,
        'Эконом Оптимум': 4876,
        'Эконом Премиум': 5846
        }
    # 'Бизнес': {
    #     'Бизнес Standard': 39380,
    #     'Бизнес Plus': 49390
    #     }
}
_status = 'available'

In [9]:
cursor = conn.cursor()
for flight_id in range(base_flight_id + 1, base_flight_id + 30):
    _aircraft_id = flight_id
    _seat_id = _aircraft_id * 10000
    for _category in ['Взрослый', 'Детский']:
        for _class in fares:
            for _fare_name in fares[_class]:
                for row in range(1, 31+1):
                    for seat_symb in ['A', 'B', 'C', 'D', 'E', 'F']:
                        _seat_number = str(row) + seat_symb
                        _seat_id += 1
                        _price = fares[_class][_fare_name]
                        if _category == 'Детский':
                            _price = _price - 100*int((_price * 0.25)/100)
                        cursor.execute("INSERT INTO seats (category, class, fare_name, number, seat_id, price, status, aircraft_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                        ((_category, _class, _fare_name, _seat_number, _seat_id, _price, _status, _aircraft_id)))
                        # print((_category, _class, _fare_name, _fare_name, _seat_number, _seat_id, _price, _status, _aircraft_id))
conn.commit()

UniqueViolation: ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "seats_pkey"
DETAIL:  Ключ "(seat_id)=(5507140010001)" уже существует.


In [14]:
cursor = conn.cursor()
for flight_id in range(base_flight_id + 30, base_flight_id + 60):
    _aircraft_id = flight_id
    _seat_id = _aircraft_id * 10000
    for _category in ['Взрослый', 'Детский']:
        for _class in fares:
            for _fare_name in fares[_class]:
                for row in range(1, 31+1):
                    for seat_symb in ['A', 'B', 'C', 'D', 'E', 'F']:
                        _seat_number = str(row) + seat_symb
                        _seat_id += 1
                        _price = fares[_class][_fare_name]
                        if _category == 'Детский':
                            _price = _price - 100*int((_price * 0.25)/100)
                        cursor.execute("INSERT INTO seats (category, class, fare_name, number, seat_id, price, status, aircraft_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                        ((_category, _class, _fare_name, _seat_number, _seat_id, _price, _status, _aircraft_id)))
                        # print((_category, _class, _fare_name, _fare_name, _seat_number, _seat_id, _price, _status, _aircraft_id))
conn.commit()

In [6]:
fare_names = ['Минимум', 'Оптимум', 'Премиум']
_category = 'Взрослый'
_class = 'Эконом'
_aircraft_id = 45465
_fare_name = 'Минимум'
_status = 'available'
prices = {
    'Минимум': 1495,
    'Оптимум': 2795,
    'Премиум': 6495,
    'Бизнес': 25185
}

In [31]:
cursor = conn.cursor()
for _aircraft_id in range(45466, 45472 + 1):
    _seat_id = _aircraft_id * 10000
    for _category in ['Взрослый', 'Детский']:
        for fare_name in fare_names:
            for row in range(1, 31+1):
                if row == 1:
                    _class = 'Бизнес'
                    _fare_name = 'Бизнес'
                else:
                    _class = 'Эконом'
                    _fare_name = fare_name
                for seat_symb in ['A', 'B', 'C', 'D', 'E', 'F']:
                    _seat_number = str(row) + seat_symb
                    _seat_id += 1
                    _price = prices[_fare_name]
                    if _category == 'Детский':
                        _price = prices[_fare_name] - 100*int((prices[_fare_name] * 0.25)/100)
                    cursor.execute("INSERT INTO seats (category, class, fare_name, number, seat_id, price, status, aircraft_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                    ((_category, _class, _fare_name, _seat_number, _seat_id, _price, _status, _aircraft_id)))
                    # print((_category, _class, _fare_name, _fare_name, _seat_number, _seat_id, prices[_fare_name], _status, _aircraft_id))

In [32]:
conn.commit()

In [ ]:
cursor = conn.cursor()
for index, row in airlines_table.iterrows():
    cursor.execute("INSERT INTO seats (category, class, fare_name, number, seat_id, price, status, aircraft_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
    (row['category'], row['class'], row['fare_name'], row['number'], row['seat_id'], row['price'], row['status'], row['aircraft_id']))
conn.commit()

In [2]:
from pyflightdata import FlightData
f=FlightData()
f.login('karrauln@yandex.ru','DpW-W75-csw-py2')